# Autodeploy example with a single app

### TODO

- Comparison with mlflow
- table
- helping in a data analysis

# Setup

In [8]:
# %load_ext autoreload
# %autoreload 2

from autodeploy import setup


app_dir = '/home/guess/Desktop/autodeploy/examples/keras_sklearn'
workflow = {'gathering': 'gathering.py',
            'preprocessing': 'preprocessing.py',
            'modeling': 'modeling.py',
            'tuning': None,
            'main': 'main.py'}

platform = setup.Setup(app_dir, workflow)
platform

In [9]:
platform.pipeline()
platform

01-Oct-19 09:15:52 - Pipeline: build() and run() are running.
01-Oct-19 09:15:52 - Dockerfile was found.
01-Oct-19 09:15:52 - Image app_single was built successfully.
01-Oct-19 09:15:52 - MLproject was found.
01-Oct-19 09:15:53 - Image app_single is running as app_single container.
01-Oct-19 09:15:53 - MLflow server is running at 0.0.0.0:8001


In [7]:
# platform.stop()

01-Oct-19 09:15:15 - Container app_single was stopped.
01-Oct-19 09:15:15 - Stopped containers were deleted.


# Deploy

In [10]:
from autodeploy import deploy

deployer = deploy.Deploy(platform)

deployer.pipeline()

01-Oct-19 09:15:58 - Pipeline: run_workflow() and deploy() are running.
01-Oct-19 09:15:58 - Running workflow: type=single .
01-Oct-19 09:16:03 -  Main file (main.py) output:  No matching run has been found.
2019/10/01 07:15:59 INFO mlflow.projects: === Created directory /tmp/tmpen4w8g9t for downloading remote URIs passed to arguments of type 'path' ===
2019/10/01 07:15:59 INFO mlflow.projects: === Running command 'python gathering.py' in run with ID 'e7229730bdea4bf88266288ab566f6fb' === 
1.2.0
        x_1       x_2       x_3       x_4  y
0 -1.887365 -1.145569  0.839676 -2.008856  0
1 -0.182668 -0.122267  0.082513 -0.205466  0
2 -0.731595  0.655904  0.205311  0.287141  0
3 -0.774965  0.744027  0.212103  0.351875  0
4 -1.339423 -1.042463  0.620971 -1.647999  0
2019/10/01 07:15:59 INFO mlflow.projects: === Run (ID 'e7229730bdea4bf88266288ab566f6fb') succeeded ===
No matching run has been found.
2019/10/01 07:15:59 INFO mlflow.projects: === Created directory /tmp/tmpgwemvohv for download

#### TODO

- Change position of mlruns (artifacts)
- Add info about port deployed (50001)


## Track

- Replication when using several containers

In [1]:
from autodeploy.deploy import deploy
from autodeploy.track import track

deployer = deploy.Deploy()
deployer.run_api()

tracker = track.Track()

sample_input = { 
        "columns": [ 
            "x_1", 
            "x_2", 
            "x_3", 
            "x_4", 
        ], 
        "data": [ 
            [-1.8873649,-1.1455,0.83967,-2.008855],
            [-0.989919,  0.987401, 0.266892,  0.485329]  
        ] 
}   

predictions = tracker.predict(sample_input)
print(f'Predictions {predictions}')

06-Oct-19 16:23:35 -  Creating container: app_single_api. 
06-Oct-19 16:23:37 - API model is running as app_single_api container.
06-Oct-19 16:23:37 -  Container: app_single_api was created successfully. 
06-Oct-19 16:23:37 -  API at: htpp://localhost:5001. 


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

### Gather a sample from the training data and new distribution


It aims at checking the sanity of new data; basically, test integrity (same numbers of features, class labels, types, etc.) and draft distributions (tests, e.g. Kolmogorov.).

In [9]:
import numpy as np
import pandas as pd
import pandas_profiling

df = pd.DataFrame(
    np.random.rand(100, 5),
    columns=['a', 'b', 'c', 'd', 'e']
)

profile = df.profile_report()


In [13]:
profile = profile.get_description()
profile['table']

{'n': 100,
 'nvar': 5,
 'memsize': 4080,
 'recordsize': 40.8,
 'n_cells_missing': 0,
 'n_vars_with_missing': 0,
 'n_vars_all_missing': 0,
 'p_cells_missing': 0.0,
 'n_duplicates': 0,
 'p_duplicates': 0.0,
 'CAT': 0,
 'BOOL': 0,
 'NUM': 5,
 'DATE': 0,
 'URL': 0,
 'PATH': 0,
 'CONST': 0,
 'UNIQUE': 0,
 'UNSUPPORTED': 0,
 'CORR': 0,
 'RECODED': 0,
 'REJECTED': 0}

In [ ]:
from autodeploy import tools

X_new = sample_input

assert integrity_checker(X_old) == integrity_checker(X_new)
tracker.overall_test(X_old, X_new)

True

False


## Test data augmentation: generate new feedbacks

In [ ]:

new_feedbacks = tracker.generate_feedbacks(feedbacks)

augmented_data = X_old+new_feedbacks
model.train(augmented_data)

new_predictions = model.predict(X_hidden)
print(model.evaluate(new_predictions , y_hidden))


### If the new model is better then the data augmentation has worked succesfully.

Old score = 0.81

New score = 0.83

#### TODO

- Show predictions in a better way.